# Проектирование экспериментов

## Введение в проектирование экспериментов

In [13]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
from statsmodels import stats as sms

import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

#Загрузка данных
df = pd.read_csv('data/KaggleV2-May-2016.csv')
display(df.head())


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [14]:
#Определим соотношение людей которые пропускают приемы,
#к тем, кто этого не делает с помощью гистограммы

fig = px.histogram(
    data_frame=df,
    x='No-show',
    title= 'Соотношение людей, которые пропускают приемы, к тем, кто этого не делает',
    width=1000,
    height=600
)
fig.show()

In [15]:
#Или можно получить точную пропорцию при помощи метода value_counts()
df['No-show'].value_counts(True)

No-show
No     0.798067
Yes    0.201933
Name: proportion, dtype: float64

In [16]:
#При помощи этого метода определим, кто чаще не является на прием - мужчины или женщины
df[df['No-show'] == 'Yes']['Gender'].value_counts(True)

Gender
F    0.653882
M    0.346118
Name: proportion, dtype: float64

Итак мы видим, что людей, пропускающих прием, примерно 20%. Из них около 65% женщины.

In [17]:
#Для обучения модели машинного обучения, необходимо закодировать столбцы
#'Gender' и 'Neighbourhood', так как для ММО необходимы числовые признаки.
categorical_columns_names = ['Gender', 'Neighbourhood']
encoded_columns = pd.get_dummies(df, columns= categorical_columns_names)

## Логирование экспериментов

In [18]:
#Импортирование модуля logging для создания логирования
import logging

### Настройка уровня логирования

In [19]:
#Установка уровня логирования
import logging
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('This message is a warning')
logging.error('This is an error message')
logging.critical('This is a critical message')

#Тут теперь отображаются тоже все 5 уровней, до этого было 3
#начиная с WARNING и выше

DEBUG:root:This is a debug message
INFO:root:This is an informational message
ERROR:root:This is an error message
CRITICAL:root:This is a critical message


In [20]:
#Перезапустим ноутбук чтобы сбросить настройки logging
      #перезапустили
#Изменим конфигурацию логгера, чтобы вывести все урвони
import logging
logging.basicConfig(level=logging.DEBUG)

#Установим уровни логирования
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('This message is a warning')
logging.error('This is an error message')
logging.critical('This is a critical message')

DEBUG:root:This is a debug message
INFO:root:This is an informational message
ERROR:root:This is an error message
CRITICAL:root:This is a critical message


In [21]:
#Другой вариант сбросить настройки logging
#воспользоваться функцией reload()
from importlib import reload
import logging
reload(logging)

#Изменим конфигурацию логгера, чтобы вывести все урвони
logging.basicConfig(level=logging.DEBUG)

#Установим уровни логирования
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('This message is a warning')
logging.error('This is an error message')
logging.critical('This is a critical message')

DEBUG:root:This is a debug message
INFO:root:This is an informational message
ERROR:root:This is an error message
CRITICAL:root:This is a critical message


### Установка обработчика (хендлера) логирования

In [22]:
#Воспользуемся обработчиком FileHandler, который возьмет запись журнала
#и добавит его в файл журнала
logging.FileHandler('log_file.log')

<FileHandler c:\Program Files (x86)\IDE\Skillfactory\EDA_5_Проектирование экспериментов\log_file.log (NOTSET)>

### Установка формата логирования

In [23]:
#Так как мы переопределяем настройки библиотеки logging,
#необходимо сбросить конфигурацию
from importlib import reload
import logging
reload(logging)
#Установим свой формат логирования
logging.basicConfig(
    format="%(levelname)s: %(asctime)s: %(message)s",
    level=logging.DEBUG
)
logging.info('Check')

INFO: 2025-07-24 19:46:31,499: Check


## Знакомство с Comet.ml